In [2]:
MYPROJECT = '$HOME/Code/mpatlas-dj2/mpatlas'
import os, sys
sys.path.insert(0, MYPROJECT)
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "settings.py")
import django
django.setup()

# Convert Aldryn_Style to DjangoCMS_Style

In [533]:
from cms.models import CMSPlugin
from aldryn_style.models import Style
from aldryn_style.cms_plugins import StylePlugin
from djangocms_style.models import Style as DStyle
from djangocms_style.cms_plugins import StylePlugin as DStylePlugin
from cms.utils.plugins import copy_plugins_to_placeholder

In [567]:
def migrateStyle(old_plugin):
    old_plugin = old_plugin.get_bound_plugin()
    children = old_plugin.get_children()
    
    new_plugin = DStyle(
        language = old_plugin.language,
        parent = old_plugin.parent,
        placeholder = old_plugin.placeholder,
        position = old_plugin.position,
        plugin_type = 'DjangoCMSStylePlugin',

        label = old_plugin.label,
        class_name = old_plugin.class_name,
        id_name = old_plugin.id_name,
        tag_type = old_plugin.tag_type,
        additional_classes = old_plugin.additional_class_names,
        padding_top = old_plugin.padding_top,
        padding_right = old_plugin.padding_right,
        padding_bottom = old_plugin.padding_bottom,
        padding_left = old_plugin.padding_left,
        margin_top = old_plugin.margin_top,
        margin_right = old_plugin.margin_right,
        margin_bottom = old_plugin.margin_bottom,
        margin_left = old_plugin.margin_left,
    )

    # insert new plugin into tree at original position, shifting original to right
    new_plugin = old_plugin.add_sibling(pos='left', instance=new_plugin)

    # This does not seem to stick for some reason
    #for child in children:
    #    child.move(target=new_plugin, pos='last-child')
    
    new_plugin = new_plugin.__class__.objects.get(pk=new_plugin.pk)
    old_plugin = old_plugin.__class__.objects.get(pk=old_plugin.pk)
    copy_plugins_to_placeholder(plugins=old_plugin.get_descendants(), placeholder=old_plugin.placeholder, root_plugin=new_plugin)
    
    new_plugin.copy_relations(old_plugin)
    new_plugin.post_copy(old_plugin, [(new_plugin, old_plugin),])
    
    # in case this is a child of a TextPlugin that needs
    # its content updated with the newly copied plugin
    if new_plugin.parent:
        new_plugin.parent.post_copy(new_plugin.parent, [(new_plugin, old_plugin),])
        
    old_plugin.delete()

    return new_plugin

In [568]:
for style in Style.objects.all():
    new_plugin = migrateStyle(style)

In [577]:
dstyles = CMSPlugin.objects.filter(plugin_type='DjangoCMSStylePlugin')
for ds in dstyles:
    ds.plugin_type = 'StylePlugin'
    ds.save()
CMSPlugin.objects.filter(plugin_type='DjangoCMSStylePlugin').count()

8

In [576]:
CMSPlugin.objects.filter(plugin_type='DjangoCMSStylePlugin').update(plugin_type='StylePlugin')
CMSPlugin.objects.filter(plugin_type='DjangoCMSStylePlugin').count()

0

# Convert cmsplugin_filer_image to djangocms_picture

In [771]:
from cms.models import CMSPlugin
from cmsplugin_filer_image.models import FilerImage
from djangocms_picture.models import Picture
from cms.utils.plugins import copy_plugins_to_placeholder

In [774]:
def migrateImage(old_plugin):
    old_plugin = old_plugin.get_bound_plugin()
    
    link = ''
    # priority was free_link > page_link > file_link > original file
    if not old_plugin.free_link and old_plugin.file_link:
        link = old_plugin.file_link.url
    elif not old_plugin.free_link and old_plugin.original_link:
        if old_plugin.image:
            link = old_plugin.image.url
        else:
            link = old_plugin.image_url
    
    new_plugin = Picture(
        language = old_plugin.language,
        parent = old_plugin.parent,
        placeholder = old_plugin.placeholder,
        position = old_plugin.position,
        plugin_type = 'PicturePlugin',

        # copy fields
        template = 'default',
        # attributes = '',
        picture = old_plugin.image,
        external_picture = old_plugin.image_url or '',
        width = old_plugin.width,
        height = old_plugin.height,
        alignment = old_plugin.alignment or '',
        caption_text = old_plugin.caption_text or '',
        link_url = link,
        link_page = old_plugin.page_link,
        link_target = '_blank' or '',
        link_attributes = old_plugin.link_attributes,
        use_automatic_scaling = old_plugin.use_autoscale,
        use_no_cropping = old_plugin.use_original_image,
        use_crop = old_plugin.crop,
        use_upscale = old_plugin.upscale,
        thumbnail_options = old_plugin.thumbnail_option,
        
        # style, # use attributes instead
        # alt_text, # not used
    )

    # insert new plugin into tree at original position, shifting original to right
    new_plugin = old_plugin.add_sibling(pos='left', instance=new_plugin)
    
    # new_plugin.copy_relations(old_plugin)
    new_plugin.post_copy(old_plugin, [(new_plugin, old_plugin),])
    
    # in case this is a child of a TextPlugin that needs
    # its content updated with the newly copied plugin
    plugin_replacements = []
    parent = new_plugin.parent
    if parent:
        # we need a complete list of (new, old) plugin pairs
        # for all children, not just the one we are replacing,
        # otherwise all plugin references will get blanked out
        # in TextPlugins
        parent = parent.get_bound_plugin();
        for child in parent.get_children():
            if child.pk != new_plugin.pk:
                if child.pk == old_plugin.pk:
                    replacement = new_plugin # use the new plugin instead of old
                else:
                    replacement = child # keep this as is
                plugin_replacements.append((replacement, child))
        parent.post_copy(parent, plugin_replacements)
        
    old_plugin.delete()

    return new_plugin

In [775]:
qs = FilerImage.objects.filter(pk=16184)
for image in qs:
    new_plugin = migrateImage(image)

In [776]:
new_plugin = new_plugin.__class__.objects.get(pk=new_plugin.pk)
old_plugin = FilerImage.objects.get(pk=16184)
print(new_plugin.pk)
print(old_plugin.pk)

DoesNotExist: FilerImage matching query does not exist.

In [759]:
qs = Picture.objects.all()
qs._raw_delete(qs.db)
qs = CMSPlugin.objects.filter(plugin_type='PicturePlugin')
qs._raw_delete(qs.db)

0

In [777]:
print('Pictures:', Picture.objects.all().count())
print('CMSPlugin PicturePlugins:', CMSPlugin.objects.filter(plugin_type='PicturePlugin').count())

Pictures: 1
CMSPlugin PicturePlugins: 1


In [769]:
new_plugin.parent.get_bound_plugin().body

'<p>Hi there\xa0<cms-plugin id="16173" title="Link - Google" alt="Link - Google "></cms-plugin></p>\n\n<p><cms-plugin id="16183" title="Image - mpatlas_logo_blue_100pix.jpg" alt="Image - mpatlas_logo_blue_100pix.jpg "></cms-plugin></p>'